In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from os.path import join
import matplotlib.pyplot as plt
from keras.models import load_model
import pickle
from sklearn  import metrics
from PIL import Image
from keras import optimizers
from keras import layers



Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers.core import Dense,Dropout,Activation
from keras.layers import Conv2D
from keras.layers.pooling import AveragePooling2D,GlobalAveragePooling2D
from keras.layers import Input,Concatenate
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import RMSprop,Adamax
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,LearningRateScheduler,EarlyStopping

In [3]:

def conv_layer(x,concat_axis,nb_filter,dropout_rate=None,weight_decay=1E-4):
    x = BatchNormalization(axis=concat_axis,
                          gamma_regularizer=l2(weight_decay),
                          beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter,(3,3),padding='same',kernel_regularizer=l2(weight_decay),use_bias=False)(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    return x

In [4]:
def transition_layer(x,concat_axis,nb_filter,dropout_rate=None,weight_decay=1E-4):
    x = BatchNormalization(axis=concat_axis,
                          gamma_regularizer=l2(weight_decay),
                          beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter,(1,1),padding='same',kernel_regularizer=l2(weight_decay),use_bias=False)(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((2,2),strides=(2,2))(x)
    return x

In [5]:
def denseblock(x,concat_axis,nb_filter,nb_layers,growth_rate,dropout_rate=None,weight_decay=1E-4):
    list_features = [x]
    for i in range(nb_layers):
        x = conv_layer(x,concat_axis,growth_rate,dropout_rate=None,weight_decay=1E-4)
        list_features.append(x)
        x = Concatenate(axis=concat_axis)(list_features)
        nb_filter += growth_rate
    return x,nb_filter

In [6]:
def Densenet(nb_classes,img_dim,depth,nb_dense_block,nb_filter,growth_rate,
             dropout_rate=None,weight_decay=1E-4):
    if K.image_dim_ordering() == "th":
        concat_axis = 1
    elif K.image_dim_ordering() == "tf":
        concat_axis = -1
        
    model_input = Input(shape=img_dim)
    
    assert (depth-4)%3  == 0 , "Depth must be 4*N +3"
    
    nb_layers = int((depth-4 )/ 3) 
    
    x = Conv2D(nb_filter,(3,3),padding='same',use_bias=False,
               kernel_regularizer=l2(weight_decay))(model_input)
    
    for block_id in range(nb_dense_block-1):
        
        x,nb_filter = denseblock(x,concat_axis,nb_filter,nb_layers,growth_rate,
                                 dropout_rate=None,weight_decay=1E-4)
        x = transition_layer(x,concat_axis,nb_filter,dropout_rate=None,weight_decay=1E-4)
        
    x = BatchNormalization(axis=concat_axis,
                          gamma_regularizer=l2(weight_decay),
                          beta_regularizer=l2(weight_decay))(x)
    
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(data_format=K.image_data_format())(x)
    
    x = Dense(nb_classes,activation='softmax',kernel_regularizer=l2(weight_decay),
                            bias_regularizer=l2(weight_decay))(x)
    densenet = Model(inputs=[model_input], outputs=[x], name="DenseNet")
    
    return densenet
    

In [8]:
 model =         Densenet(nb_classes=2,
                          img_dim=(112,112,1),
                          depth = 34,
                          nb_dense_block = 2,
                          growth_rate=12,
                          nb_filter=32,
                          dropout_rate=0.25,
                          weight_decay=1E-4)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 112, 112, 1)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 112, 112, 32) 288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 112, 112, 32) 128         conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 112, 112, 32) 0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
conv2d_3 (

In [9]:

model.compile(loss='categorical_crossentropy',
              optimizer = Adamax(),
              metrics=["accuracy"])

In [10]:
#MODEL_TO_EVAL = '/Users/sidgupta/Downloads/NASNetMobile.hdf5'
DATA_DIR = '/Users/sidgupta/Documents/mura'
#EVAL_CSV = 'valid.csv'
data_dir = DATA_DIR


EVAL_DIR = join(data_dir, 'humerus')
PRED_DIR = join(data_dir, 'humerus_v')

In [11]:
def preprocess(img):
   width, height = img.shape[0], img.shape[1]
   img = image.array_to_img(img, scale=False)

   # Crop 48x48px
   desired_width, desired_height = 48, 48
   """
   if width < desired_width:
       desired_width = width"""
   #start_x = np.maximum(0, int((width-desired_width)/2))
   img=img.convert("L")
   #img = img.crop((start_x, np.maximum(0, height-desired_height), start_x+desired_width, height))
   img = img.resize((112, 112),Image.ANTIALIAS)
   #img = img.reshape(112,112,1)
   img = image.img_to_array(img)
   return img / 255.

In [12]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
preprocessing_function=preprocess)
 
validation_datagen = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess)
 
# Change the batchsize according to your system RAM
train_batchsize = 200
val_batchsize = 20
 
train_generator = train_datagen.flow_from_directory(
        EVAL_DIR,
        target_size=(112, 112),
        batch_size=train_batchsize,
        class_mode='categorical',
color_mode='grayscale')
 
validation_generator = validation_datagen.flow_from_directory(
        PRED_DIR,
        target_size=(112, 112),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,color_mode='grayscale')

Found 1272 images belonging to 2 classes.
Found 288 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)
 
with open('densenet_.pkl', 'wb') as pickle_out:  
    pickle.dump(history, pickle_out)    
# Save the model
model.save('densenet_.h5')

Epoch 1/30
1/6 [===>..........................] - ETA: 1:44:22 - loss: 1.1814 - acc: 0.4800